<a href="https://colab.research.google.com/github/jahelsantiago/tensorflow_tutorial/blob/master/IMBD_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2020-08-24 02:12:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.05s   

2020-08-24 02:12:02 (106 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [3]:
import pandas as pd

df_data = pd.read_csv(r"/tmp/bbc-text.csv")
df_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [4]:
sentenes = df_data["text"].values
labels = df_data["category"].values

In [25]:
train_part = 0.8
vocab_size = 100000 #num of the words in the dictionary 
oov_tok = "<OOV>"
max_length = 120
trunc_type = "post"
padding_type = "post"
embeding_dim = 300

In [8]:
train_size = int(len(sentenes)*train_part)

train_sentences = sentenes[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentenes[train_size:]
validation_labels = labels[train_size:]

print(len(train_sentences))
print(len(validation_sentences))
print(train_sentences[0])

1780
445
tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import  pad_sequences

tokeizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokeizer.fit_on_texts(train_sentences)
word_index = tokeizer.word_index

train_sequences = tokeizer.texts_to_sequences(train_sentences)
train_sequences_padded = pad_sequences(train_sequences, max_length, padding= padding_type, truncating= trunc_type)

validation_sequence = tokeizer.texts_to_sequences(validation_sentences)
validation_sequence_padded = pad_sequences(validation_sequence, max_length, padding= padding_type, truncating= trunc_type)

In [26]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

train_labels_seq = label_tokenizer.texts_to_sequences(train_labels)
train_labels_seq_pad = pad_sequences(train_labels_seq, padding=padding_type, truncating=trunc_type)

validation_labels_seq = label_tokenizer.texts_to_sequences(validation_labels)
validation_labels_seq_pad = pad_sequences(validation_labels_seq, padding=padding_type, truncating=trunc_type)

label_word_index = label_tokenizer.word_index

In [43]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential

model = Sequential([
  Embedding(vocab_size, embeding_dim, input_length= max_length),
  GlobalAveragePooling1D(),
  Dense(24, activation="relu"),
  Dense(6, activation="softmax")
])

model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 120, 300)          30000000  
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 24)                7224      
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 150       
Total params: 30,007,374
Trainable params: 30,007,374
Non-trainable params: 0
_________________________________________________________________


In [47]:
num_epochs = 10
histoy = model.fit(train_sequences_padded ,train_labels_seq_pad, epochs=num_epochs, validation_data=(validation_sequence_padded,validation_labels_seq_pad))

Epoch 1/10
56/56 [==============================] - 16s 291ms/step - loss: 0.7634 - accuracy: 0.9213 - val_loss: 0.5728 - val_accuracy: 0.9438
Epoch 2/10
56/56 [==============================] - 16s 291ms/step - loss: 0.2832 - accuracy: 0.9882 - val_loss: 0.3097 - val_accuracy: 0.9483
Epoch 3/10
56/56 [==============================] - 16s 291ms/step - loss: 0.1097 - accuracy: 0.9933 - val_loss: 0.2259 - val_accuracy: 0.9506
Epoch 4/10
56/56 [==============================] - 16s 291ms/step - loss: 0.0524 - accuracy: 0.9983 - val_loss: 0.1921 - val_accuracy: 0.9506
Epoch 5/10
56/56 [==============================] - 16s 292ms/step - loss: 0.0283 - accuracy: 0.9994 - val_loss: 0.1724 - val_accuracy: 0.9506
Epoch 6/10
56/56 [==============================] - 16s 289ms/step - loss: 0.0171 - accuracy: 1.0000 - val_loss: 0.1632 - val_accuracy: 0.9528
Epoch 7/10
56/56 [==============================] - 16s 289ms/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.1573 - val_accuracy: 0.9551

In [48]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: ignored